In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

In [ ]:
# Carregando os dados
df = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')
entrega = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/sample_submission.csv')

df.shape, test.shape, entrega.shape

In [ ]:
# Juntando os dataframes
df_all = df.append(test)

df_all.shape

In [ ]:
# Vamos transformar 'yes' em 1 e 'no' em 0
# nas colunas edjefa e edjefe
mapeamento = {'yes': 1, 'no': 0}

In [ ]:
df_all['edjefa'] = df_all['edjefa'].replace(mapeamento).astype(int)
df_all['edjefe'] = df_all['edjefe'].replace(mapeamento).astype(int)

#### Fiz o mesmo que foi realizado em aula com o dataset de treino

In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

In [ ]:
# Vamos transformar 'yes' em 1 e 'no' em 0
# na coluna dependency
df_all['dependency'] = df_all['dependency'].replace(mapeamento).astype(float)

In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

In [ ]:
 # Verificando os valores de aluguel (v2a1) para os chefes/as de familia (parentesco1 = 1)
df_all[df_all['parentesco1'] == 1]['v2a1'].isnull().sum()

In [ ]:
# Prenchendo com -1 os valores nulos de v2a1
df_all['v2a1'].fillna(-1, inplace=True)

In [ ]:
df['v2a1'].fillna(-1, inplace=True)

In [ ]:
# Prenchendo com 0 os valores nulos de v18q1
df_all['v18q1'].fillna(0, inplace=True)

#### A maioria das variáveis com missing values foram substituídas por -1, pois não podemos assumir que os valores ausentes são 0. Eles vão interferir nos dados. Já a variável 'v18q1' é a quantidade de tablets dentro de uma residência. Comparando ela com a variável v18q, podemos concluir que os valores ausentes são as residências que não possuem tablets. Logo, esses valores ausentes podem ser substituídos por 0.

In [ ]:
# Verificando os valores nulos
df_all.isnull().sum().sort_values()

In [ ]:
# Prenchendo com -1 os valores nulos de SQBmeaned, meaneduc e rez_esc
df_all['SQBmeaned'].fillna(-1, inplace=True)
df_all['meaneduc'].fillna(-1, inplace=True)
df_all['rez_esc'].fillna(-1, inplace=True)

### Análise Exploratória de algumas variáveis

##### Para as variáveis binárias, eu realizei um agrupamento delas com o total de entradas de cada variável, vendo a distribuição delas em histogramas

In [ ]:
pared = df_all.filter(regex='^pared',axis=1)

pared_sum = pared.sum()

parede = pared_sum.to_frame()

nomes_paredes = ['Concreto ou tijolo', 'Parede revestida', 'Pré-fabricada ou cimento', 
                 'Material reciclado', 'Madeira', 'Zinco', 'Fibras naturais', 'Outros']

pared_sum.index = nomes_paredes

plt.figure(figsize=(15,10))

plt.bar(pared_sum.index.values, pared_sum)

In [ ]:
ec = df_all.filter(regex='^estadocivil',axis=1)

ec = ec.sum()

nomes_ec = ['Menos de 10 anos de idade', 'União estável', 'Casado', 
            'Divorciado', 'Separado', 'Viúvo(a)', 'Solteiro']

ec.index = nomes_ec

plt.figure(figsize=(15,10))

plt.bar(ec.index.values, ec)

In [ ]:
lixo = df_all.filter(regex='^elimbasu',axis=1)

lixo = lixo.sum()

nomes_lixo = ['Caminhão de lixo', 'Buraco ou enterrado', 'Cremado', 
              'Espaço não ocupado', 'Jogado em rio, lagoa ou mar', 'Outro']
lixo.index = nomes_lixo

plt.figure(figsize=(15,10))

plt.bar(lixo.index.values, lixo)

In [ ]:
sanitario = df_all.filter(regex='^sanitario',axis=1)

sanitario = sanitario.sum()

nomes_sanitario = ['Sem sistema sanitário', 'Esgoto', 'Tanque séptico', 'Latrina', 'Outro sistema']
sanitario.index = nomes_sanitario

plt.figure(figsize=(15,10))

plt.bar(sanitario.index.values, sanitario)

In [ ]:
agua = df_all.filter(regex='^abastagua',axis=1)

agua = agua.sum()

nomes_agua = ['Abastecimento interno', 'Abastecimento externo', 'Sem água']
agua.index = nomes_agua

plt.figure(figsize=(10,10))

plt.bar(agua.index.values, agua)

In [ ]:
teto = df_all.filter(regex='^techo',axis=1)

teto = teto.sum()

nomes_teto = ['Folha laminada ou zinco', 'Fibra de cimento, mezanino', 'Fibras naturais', 'Outros']

teto.index = nomes_teto

plt.figure(figsize=(15,10))

plt.bar(teto.index.values, teto)

In [ ]:
piso = df_all.filter(regex='^piso',axis=1)

piso = piso.sum()

nomes_piso = ['Mosaico, cerâmica ou porcelanato', 'Cimento', 'Outro', 'Material Natural', 'Sem piso', 'Madeira']

piso.index = nomes_piso

plt.figure(figsize=(15,10))

plt.bar(piso.index.values, piso)

In [ ]:
parentesco = df_all.filter(regex='^parentesco',axis=1)

parentesco = parentesco.sum()

nomes_parentesco = ['Chefe da casa', 'Esposo/parceiro', 'Filho(a)',
                    'Enteado(a)', 'Noro(a)', 'Neto(a)', 'Mãe/pai', 
                    'Sogro(a)', 'Irmã(o)', 'Cunhado(a)', 
                    'Outro parente', 'Outra pessoa']

parentesco.index = nomes_parentesco

plt.figure(figsize=(20,10))

plt.bar(parentesco.index.values, parentesco)

In [ ]:
escol = df_all.filter(regex='^instlevel',axis=1)

escol = escol.sum()

nomes_escol = ['Sem educação', 'Primário incompleto', 'Primário completo', 
               'Secundário incompleto', 'Secundário completo', 'Técnico incompleto',
               'Técnico completo', 'Ensino superior', 'Pós-graduação']
               
escol.index = nomes_escol

plt.figure(figsize=(20,10))

plt.bar(escol.index.values, escol)

In [ ]:
casa = df_all.filter(regex='^tipovivi',axis=1)

casa = casa.sum()

nomes_casa = ['Casa própria', 'Casa própria, mas pagando', 'Alugada', 'Precária', 'Outro']

casa.index = nomes_casa

plt.figure(figsize=(15,10))

plt.bar(casa.index.values, casa)

In [ ]:
lugar = df_all.filter(regex='^lugar',axis=1)

lugar = lugar.sum()

nomes_lugar = ['Central', 'Chorotega', 'Pacífico Central', 'Brunca',
              'Huetar Atlántica', 'Huetar Norte']

lugar.index = nomes_lugar

plt.figure(figsize=(15,10))

plt.bar(lugar.index.values, lugar)

In [ ]:
area = df_all.filter(regex='^area',axis=1)

area = area.sum()

nomes_area = ['Área urbana', 'Área Rural']

area.index = nomes_area

plt.figure(figsize=(5,10))

plt.bar(area.index.values, area)

In [ ]:
df_all['Idade'] = df_all['age']

plt.figure(figsize=(20,12))
sns.histplot(data=df_all, x='Idade')
plt.show()

### Tratando os dados desbalanceados e utilizando Random Forest para a variável Target 

##### As primeiras células foram retiradas do notebook criado na aula. As células seguintes foram baseadas nesse notebook e em outras submissões da competição.

In [ ]:
# Separando as colunas para treinamento
feats = [c for c in df_all.columns if c not in ['Id', 'idhogar', 'Target']]

In [ ]:
# Separar os dataframes
train, test = df_all[~df_all['Target'].isnull()], df_all[df_all['Target'].isnull()]

X_train, Y_train = train[feats], train[['Target']]
X_test, Y_test = test[feats], test[['Target']]

train.shape, test.shape, X_test.shape, Y_test.shape

In [ ]:
train = train.drop(['Id', 'idhogar'], axis=1)

#### As células abaixo estão como comentário para rodar apenas o último resultado

In [ ]:
# Instanciando o random forest classifier
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=42)

In [ ]:
# Treinando o modelo

#rf.fit(train[feats], train['Target'])

In [ ]:
# Prever o Target de teste usando o modelo treinado

#test['Target'] = rf.predict(test[feats]).astype(int)

In [ ]:
# Criando o arquivo para submissão
# test[['Id', 'Target']].to_csv('submission.csv', index=False)

##### A partir daqui, tentamos utilizar Over Sampling para poder tratar o desbalanceamento dos dados.

In [ ]:
ros = RandomOverSampler(random_state = 42)
X_ros, y_ros = ros.fit_resample(X_train, Y_train)

In [ ]:
rf = RandomForestClassifier(n_jobs=1, n_estimators=200, random_state=42)
rf.fit(X_ros,y_ros)

In [ ]:
# test['Target'] = rf.predict(test[feats]).astype(int)

In [ ]:
# test[['Id', 'Target']].to_csv('submission.csv', index=False)

##### Over Sampling não é a melhor maneira de lidar com os dados. Vamos tentar regressão .

In [ ]:
# from sklearn.feature_selection import SelectKBest, f_regression

In [ ]:
## a variável ID tem que ser dropada, pois causa erro na próxima tentativa

## x = train.drop(['Target', 'Id', 'idhogar'], axis=1)
## y = train.Target

In [ ]:
### seleciona as melhores K variáveis para o modelo

## variaveis = SelectKBest(score_func = f_regression, k = 80)
## novas_variaveis = variaveis.fit_transform(x,y)

In [ ]:
## var_selec = variaveis.get_support()

In [ ]:
## len(var_selec)

In [ ]:
## var_teste = train.columns
## var_teste

##### A próxima célula é repetiva, mas ela foi feita da seguinte maneira, pois o length da variável var_teste tem que ter o mesmo lenght da variável var_selec.

In [ ]:
# var_teste = list(var_teste)

# del var_teste[len(var_teste)-1]

# var_teste = list(var_teste)

# del var_teste[len(var_teste)-1]

# var_teste = list(var_teste)

# del var_teste[len(var_teste)-1]

In [ ]:
# len(var_teste)

In [ ]:
# var = []
# for i in range (len(var_teste)):
  #  if var_selec[i] == True:
   #     var.append(var_teste[i])

In [ ]:
# teste = train[var]

In [ ]:
# var.append('Target')

In [ ]:
# treino = train[var]

##### Aqui eu realizei o mesmo drop que foi realizado acima para poder rodar as bases de teste e treino durante o predict.

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_val, y_train, y_val = train_test_split (test, test.Target, test_size = 0.3, random_state = 42)

##### Vamos tentar utilizar regressão para conseguir as variáveis Target.

In [ ]:
# from sklearn.ensemble import RandomForestRegressor 

# florestarandomica = RandomForestRegressor(n_estimators = 200, criterion='mse', random_state=42, max_depth = 15)

In [ ]:
# florestarandomica.fit(X_train, y_train)

In [ ]:
# y_pred = florestarandomica.predict(X_val)

# from sklearn.metrics import mean_squared_error

# mean_squared_error(y_pred,y_val)

In [ ]:
# tentativa1 = florestarandomica.predict(teste)

In [ ]:
# entrega['Target'].astype(int)

In [ ]:
# entrega.to_csv('submission.csv', index=False)

##### O score obtido através dessa tentativa é menor que 0.20. Vamos tentar mudar a maneira como geramos os dados da variável Target. Vamos tentar Under Sampling.

In [ ]:
rus = RandomUnderSampler(random_state = 42)
X_under, y_under = rus.fit_resample(X_train, Y_train)

In [ ]:
rf = RandomForestClassifier(n_jobs=1, n_estimators=200, random_state=42)
rf.fit(X_under,y_under)

In [ ]:
test['Target'] = rf.predict(test[feats]).astype(int)

In [ ]:
test[['Id', 'Target']].to_csv('submission.csv', index=False)

##### Aparentemente, o Under Sample é melhor do que regressão e que Over Sample para o problema.